In [ ]:
import insightface
from insightface.app.common import Face
from insightface.model_zoo import model_zoo
from neural_nets import MMFace, insightface_model, InsightFaceClassifier, IntermediateFusionClassifier
from utils import load_model
import numpy as np
# REQUIRED FOR CUDA TO BE USED
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_subjects = 21

det_model = model_zoo.get_model("../models/buffalo_l/det_10g.onnx")
det_model.prepare(ctx_id=0, input_size=(480, 640), det_thres=0.5)
rec_model = model_zoo.get_model("../models/buffalo_l/w600k_r50.onnx")

mmface_model = MMFace(2).to(device)
mmface_model.load_state_dict(torch.load(f"models/mmFace-liveness-99.6.pt")["model_state_dict"])
mmface_model.eval()
# TODO: MAKE READY FOR FEATURE EXTRACTION LAST LAYER

insightface_classifier = InsightFaceClassifier(num_subjects).to(device)
insightface_classifier.load_state_dict(torch.load("models/insightface_classifier.pt")["model_state_dict"])
insightface_classifier.eval()

# TODO: TRAIN (ONCE PAPER DATASET COLLECTED)
insightface_classifier_liveness = InsightFaceClassifier(2).to(device)
insightface_classifier_liveness.eval()
#
# mmface_classifier_liveness = MMFaceClassifier_Liveness().to(device)
# mmface_classifier_liveness.load_state_dict(torch.load("models/mmFace-21_classifier.pt")["model_state_dict"])
# mmface_classifier_liveness.eval()

# TODO: NEED DATASET FOR THIS
intermediate_fusion_clf = IntermediateFusionClassifier(512*2, num_subjects).to(device)

# TODO: LOOP THROUGH DATASET IN ORDER AND RECOGNISE AND CLASSIFY FOR EVALUATION (LOAD DATA FOR EACH SUBJECT)
def recognise(rgb_input, radar_input):
    with torch.no_grad():
        rgb_emb = insightface_model(rgb_input[..., ::-1], det_model, Face, rec_model)
        # TODO: EXTRACT EMBEDDING FROM MODEL.FC2
        radar_emb = mmface_model(radar_input)
    
    return rgb_emb, radar_emb

def classify_no_radar(rgb_emb):
    with torch.no_grad():
        subject = insightface_classifier(rgb_emb)
        liveness = insightface_classifier_liveness(rgb_emb)

    return subject, liveness

def classify(rgb_emb, radar_emb):
    with torch.no_grad():
        subject = insightface_classifier(rgb_emb)
        liveness = mmface_classifier_liveness(radar_emb)
    
    return subject, liveness

In [31]:
import matplotlib.pyplot as plt

DATA_PATH = "../../Soli/soli_realsense/data"

def load_rgb(subject, experiment):
    return np.load(f"{DATA_PATH}/{subject}/{subject}-{experiment}_colour.npy").astype(np.float32)

# Rewrite to take embedding from files
def recognise_rgb(rgb_input):
    return insightface_model(rgb_input[..., ::-1], det_model, Face, rec_model)

x = recognise_rgb(load_rgb(0, 0)[0])
y = recognise_rgb(load_rgb(90, 10)[0])

In [32]:
score_good = np.clip(np.dot(x, y.T), 0., 1.)
# score_bad = np.clip(np.dot(fake_0, real_1.T), 0., 1.)

print(score_good, score_bad)

0.593906819820404 0.0
